# La librería gzip

Aunque la capadidad de almacenamiento en los ordenadores sigue incrementándose, también
lo hace la cantidad de detos que queremos almacenar. Para solucionar esto se inventaron
hace tiempo varios __algoritmos de compresión__, que nos permiten almacenar
la misma cantidad de información en menos espacio, teniendo que usar, eso si,un poco
más de tiempo para acceder o modificxar la informacion.

En python, Los módulos `zlib` y `gzip` nos permiten acceder a estos dos algoritmos de compresión
clásicos, mientras que el modulo `bzip2` usa un formato y algoritmo algo mås reciente.

Siempre trabajamos con un flujo de *bytes*, es decir, que podemos comprimir información en cualquier formato que nos interese, porque los algoritmos no se preocupan por el formato, solo "ven" secuencias de bytes.

Los tres módulos incluyen también funciones para trabajar, de forma transpareente, con ficheros
comprimidos.

Ademas de las funciones de compresión, habitualmente se usa también el
concepto de __archivado__, esto es, incluir varios archivos dentro de otro, normalmente
para copias de seguridad o  trasmision. 

Estos dos conceptos, compresión y archivado no son lo mismo.

- puedes tener un fichero comprimido, sin archivado (es decir, que solo
esta comprimido un unico fichero)

- O se puede tener un archivador sin usar compresión.

- Pero normalmente estas dos funcionalidades suelen ir juntas.

Nota: En la librería estándar hay módulos para trabajar con otros formatos
de compresión/archivado, como `tarfile` para trabajar con ficheros *.tar*, usados
frecuentemente en sistemas tipo Unix, o `zipfile`
para manipular archivos *.zip*, un formato muy popular en los entornos Windows
desde los tiempos del MS/DOS (Ambos sistemas pueden encontrarse ya en cualquier entorno,
no obstante). En este curso no veremos estos módulos, pero
es importante que sepan que están ahí, en caso de que los necesiten.


## El módulo zlib

Como siempre, para poder usarlo tenemos que importarlo:

In [4]:
import zlib

Podemos comprimir y descomprimir directamente contenidos en memoria. En el caso
de `zlib`, usamos los métodos __`compress`__ y __`decompress`__. Vamos a verlo ejecutando
el siguiente ejemplo.

In [21]:
import zlib

message = "Hola, me llamo Íñigo Montoya. Tu mataste a mi padre. Preparate a morir".encode("utf-8")
print(f"El mensaje origina ocupa {len(message)} bytes")
compressed = zlib.compress(message)
print(f"El mensaje comprimido ocupa {len(compressed)} bytes")

El mensaje origina ocupa 72 bytes
El mensaje comprimido ocupa 76 bytes


Ejercicio: ¿Notan algo raro en el ejemplo anterior?

![El mensaje comprimido ocupa más que el mensaje sin descomprimir](./img/emosido.jpg)

Efectivamene ¡El mensaje comprimido ocupa más que el mensaje sin descomprimir! Justo
lo contrario de lo que deberia pasar! Esto es por varias razones:

- Los algoritmos de compresión trabajan mejor cuanto mas repeticiones de símbolos haya (trabajan
    reduciendo la redundancia)

- Cuanto más contenido, mejor compresión (Más probabilidad de encontrar redundacia)

Vamos a ver que tal funciona con una supuesta secuencia de genes (Solo cuatro
símbolos, `G`, `A`, `T` y `C`):

In [34]:
import zlib

message = b"GGAAATGGTAGGGCTAGATGCCCCTTAGCTCATGCGCTGCGCTCATCAAACCTAGGTTATTAGCACTAACAT"
print(f"El mensaje original ocupa {len(message)} bytes")
compressed = zlib.compress(message)
print(f"El mensaje comprimido ocupa {len(compressed)} bytes")

El mensaje original ocupa 72 bytes
El mensaje comprimido ocupa 49 bytes


### Ejercicio: Descomprimir un mensaje

Hemo recibido el siguiente mensaje, comprimido con `zlib`:

In [40]:
message = b'x\x9c\xf3\xc9WH\xce\xcf+NM/\xcd,.IU\x04\x00,\xcc\x05\xa6'

Usa la función `decompress` del modulo `zip`.

In [42]:
import zlib

message = b'x\x9c\xf3\xc9WH\xce\xcf+NM/\xcd,.IU\x04\x00,\xcc\x05\xa6'


### Compresion/Decompresión incremental

El problema de comprimir en memoria es que nos limita, porque
necesitamos mantener a la vez los datos comprimidos
y sin descomprimir.

Para solucionar esto tenemos las clases `Compress`
y `Decompress`, que nos permiten trabajar con los datos de forma
incremental y con datos demasiado grandes para caber en memoria.



In [43]:
import zlib
import binascii

compressor = zlib.compressobj(1)

with open('lorem.txt', 'r') as input:
    while True:
        block = input.read(64)
        if not block:
            break
        compressed = compressor.compress(block)
        if compressed:
            print 'Compressed: %s' % binascii.hexlify(compressed)
        else:
            print 'buffering...'
    remaining = compressor.flush()
    print 'Flushed: %s' % binascii.hexlify(remaining)

SyntaxError: invalid syntax (<ipython-input-43-64087c225153>, line 13)

This example reads small blocks of data from a plain-text file and passes it to compress(). The compressor maintains an internal buffer of compressed data. Since the compression algorithm depends on checksums and minimum block sizes, the compressor may not be ready to return data each time it receives more input. If it does not have an entire compressed block ready, it returns an empty string. When all the data is fed in, the flush() method forces the co